In [ ]:
##Instalar pacote de requisições SOAP
import sys
!{sys.executable} -m pip install zeep

In [9]:
from zeep import Client
from zeep.wsse.username import UsernameToken

In [3]:
client = Client('https://servicoshm.saude.gov.br/cnes/EstabelecimentoSaudeService/v1r0?wsdl', wsse=UsernameToken('CNES.PUBLICO', 'cnes#2015public'))

In [4]:
##Mostrar as informações do WSDL
client.wsdl.dump()


Prefixes:
     xsd: http://www.w3.org/2001/XMLSchema
     ns0: http://servicos.saude.gov.br/cnes/v1r0/estabelecimentosaudeservice
     ns1: http://servicos.saude.gov.br/wsdl/mensageria/v1r0/filtropesquisaestabelecimentosaude
     ns2: http://servicos.saude.gov.br/schema/cnes/v1r0/codigocnes
     ns3: http://servicos.saude.gov.br/schema/corporativo/pessoajuridica/v1r0/cnpj
     ns4: http://servicos.saude.gov.br/wsdl/mensageria/v1r0/filtropesquisaprecadastrocnes
     ns5: http://servicos.saude.gov.br/wsdl/mensageria/v1r0/filtrolocalizacaoestabelecimentosaude
     ns6: http://servicos.saude.gov.br/schema/cnes/v1r0/tipounidade
     ns7: http://servicos.saude.gov.br/wsdl/mensageria/v1r0/paginacao
     ns8: http://servicos.saude.gov.br/schema/cnes/v1r0/localizacao
     ns9: http://servicos.saude.gov.br/schema/cnes/v1r0/dadosgeraiscnes
     ns10: http://servicos.saude.gov.br/schema/cnes/v1r0/codigounidade
     ns11: http://servicos.saude.gov.br/schema/corporativo/pessoajuridica/v1r0/nomejuri

In [11]:
request_data = {
    'FiltroPesquisaEstabelecimentoSaude': {
        'CodigoCNES': {
            'codigo': 2653982
        }
    },
}

In [12]:
response = client.service.consultarEstabelecimentoSaude(**request_data)

In [13]:
print (response)

{
    'CodigoCNES': {
        'codigo': '2653982'
    },
    'CodigoUnidade': {
        'codigo': '2408102653982'
    },
    'nomeFantasia': {
        'Nome': 'HUOL HOSPITAL UNIVERSITARIO ONOFRE LOPES'
    },
    'nomeEmpresarial': {
        'Nome': 'UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE'
    },
    'CNPJ': {
        'numeroCNPJ': '24365710001317'
    },
    'Endereco': {
        'identificador': None,
        'TipoEndereco': None,
        'TipoLogradouro': None,
        'nomeLogradouro': 'AV NILO PECANHA',
        'numero': '620',
        'complemento': None,
        'Bairro': {
            'codigoBairro': None,
            'descricaoBairro': 'PETROPOLIS'
        },
        'CEP': {
            'numeroCEP': '59012300'
        },
        'Municipio': {
            'codigoMunicipio': '240810',
            'nomeMunicipio': 'NATAL',
            'UF': {
                'codigoUF': '24',
                'siglaUF': 'RN',
                'codigoRegiao': None,
                'nomeUF': N

In [1]:
##Instalar pacote de requisições SOAP
import sys
!{sys.executable} -m pip install pandas

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas

In [3]:
data_frame = pandas.read_csv('listas/municipios.csv', sep=';')

In [4]:
data_frame

,MUNICIPIO,UF
0,110012,RO
1,110015,RO
2,110025,RO
3,110100,RO
4,110130,RO
5,110143,RO
6,110155,RO
7,110160,RO
8,110170,RO
9,110180,RO


In [6]:
data_frame.MUNICIPIO

0       110012
1       110015
2       110025
3       110100
4       110130
5       110143
6       110155
7       110160
8       110170
9       110180
10      110014
11      110032
12      110034
13      110150
14      110001
15      110004
16      110009
17      110028
18      110029
19      110037
20      110050
21      110090
22      110120
23      110018
24      110030
25      110092
26      110145
27      110147
28      110148
29      110003
         ...  
5540    520815
5541    521530
5542    521580
5543    521740
5544    521920
5545    522026
5546    522060
5547    522180
5548    522200
5549    520120
5550    520480
5551    520510
5552    520590
5553    520660
5554    520690
5555    520850
5556    521010
5557    521480
5558    521550
5559    522130
5560    520410
5561    520430
5562    520915
5563    521080
5564    521130
5565    521225
5566    521630
5567    521850
5568    522040
5569    530010
Name: MUNICIPIO, Length: 5570, dtype: int64

In [7]:
request_data_municipio = {
    'Municipio': {
        'codigoMunicipio' : 240810
    }
}

In [10]:
client_cnes_service = Client('https://servicos.saude.gov.br/cnes/CnesService/v1r0?wsdl', wsse=UsernameToken('CNES.PUBLICO', 'cnes#2015public'))

In [11]:
client_cnes_service.wsdl.dump()


Prefixes:
     xsd: http://www.w3.org/2001/XMLSchema
     ns0: http://servicos.saude.gov.br/cnes/v1r0/cnesservice
     ns1: http://servicos.saude.gov.br/wsdl/mensageria/v1r0/resultadopesquisadadoscomplementarescnes
     ns2: http://servicos.saude.gov.br/schema/cnes/v1r0/complementoestabelecimentosaude
     ns3: http://servicos.saude.gov.br/schema/cnes/v1r0/tipounidade
     ns4: http://servicos.saude.gov.br/schema/cnes/v1r0/tipogestao
     ns5: http://servicos.saude.gov.br/schema/cnes/v1r0/complementoprofissionalsaude
     ns6: http://servicos.saude.gov.br/schema/profissionalsaude/v1r0/vinculoempregaticio
     ns7: http://servicos.saude.gov.br/schema/profissionalsaude/v1r0/tipovinculoempregaticio
     ns8: http://servicos.saude.gov.br/schema/cnes/v1r0/complementoesferaadministrativa
     ns9: http://servicos.saude.gov.br/schema/cnes/v1r0/esferaadministrativa
     ns10: http://servicos.saude.gov.br/schema/cnes/v1r0/cmpt
     ns11: http://servicos.saude.gov.br/schema/cnes/v1r0/codigocnes

In [12]:
client_cnes_service.service.consultarEstabelecimentoSaudePorMunicipio(**request_data_municipio)

[{
     'DadosBasicosEstabelecimento': [
         {
             'CodigoCNES': {
                 'codigo': '7700490'
             },
             'NomeFantasia': {
                 'Nome': 'CONSIULTORIO MEDICO DRA ANA TERESA'
             }
         },
         {
             'CodigoCNES': {
                 'codigo': '7698127'
             },
             'NomeFantasia': {
                 'Nome': 'CONSULTORIO DE PSICOLOGIA DRA CLAUDIA MONTENEGRO'
             }
         },
         {
             'CodigoCNES': {
                 'codigo': '7707150'
             },
             'NomeFantasia': {
                 'Nome': 'INSTITUTO DO SONO E TRANSTORNOS MENTAIS'
             }
         },
         {
             'CodigoCNES': {
                 'codigo': '7769393'
             },
             'NomeFantasia': {
                 'Nome': 'VIP SAUDE HOME CARE'
             }
         },
         {
             'CodigoCNES': {
                 'codigo': '5656281'
             },
          

In [ ]:
for row in data_frame.itertuples():
    path = './resultados/%s/%s' % (row.UF, row.MUNICIPIO) 
    %mkdir (path, 0755)